In [1]:
library(dplyr)
library(Seurat)
library(patchwork)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching SeuratObject

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode



In [4]:
scRNA_path = "/data1/weiyihu/endometrium/data/raw_data"
share_path = "/data1/weiyihu/endometrium/data/share"
studyNameFile3 = c("GSE164449")
studyNameH5 = c("1.CRA002181", "3.HRA000237")
studyNameRData = c("GSE130560")

In [14]:
# 一级函数
tempRead10X = function(data.dir=NULL, barcodesFile, featuresFile, matrixFile){
  barcodesFile = barcodesFile
  featuresFile = featuresFile
  matrixFile = matrixFile
  
  full.data <- list()
  for (i in seq_along(data.dir)) {
    run <- data.dir[i]
    if (!dir.exists(run)) {
      stop("Directory provided does not exist")
    }
    if (!grepl("\\/$", run)) {
      run <- paste(run, "/", sep = "")
    }
    # 解压缩
    if(grepl(".gz$", barcodesFile)){system(paste0("gunzip ", barcodesFile)); barcodesFile = gsub(".gz$", '', barcodesFile)}
    if(grepl(".gz$", featuresFile)){system(paste0("gunzip ", featuresFile)); featuresFile = gsub(".gz$", '', featuresFile)}
    if(grepl(".gz$", matrixFile)){system(paste0("gunzip ", matrixFile)); matrixFile = gsub(".gz$", '', matrixFile)}
    
    barcode.loc <- barcodesFile
    gene.loc <- featuresFile
    matrix.loc <- matrixFile
    if (!file.exists(barcode.loc)){stop("Barcode file missing")}
    if (!file.exists(gene.loc)){stop("Gene name file missing")}
    if (!file.exists(matrix.loc)){stop("Expression matrix file missing")}
    data <- readMM(file = matrix.loc)
    cell.names <- readLines(barcode.loc)
    gene.names <- readLines(gene.loc)
    if (all(grepl(pattern = "\\-1$", x = cell.names))) {
      cell.names <- as.vector(x = as.character(x = sapply(X = cell.names, 
                                                          FUN = ExtractField, field = 1, delim = "-")))
    }
    rownames(x = data) <- make.unique(names = as.character(x = sapply(X = gene.names, 
                                                                      FUN = ExtractField, field = 2, delim = "\\t")))
    if (is.null(x = names(x = data.dir))) {
      if (i < 2) {colnames(x = data) <- cell.names}
      else {colnames(x = data) <- paste0(i, "_", cell.names)}
    }
    else {colnames(x = data) <- paste0(names(x = data.dir)[i], "_", cell.names)}
    full.data <- append(x = full.data, values = data)
  }
  full.data <- do.call(cbind, full.data)
  return(full.data)
}

# 一级函数
makeData = function(dataPath, studyName, mode, data=NULL){
  dataPath = dataPath
  studyName = studyName
  mode = mode
  
  if(is.null(data)){data=list()}
  
  # 获取每个研究中不同样本的路径
  if(mode == "3"){
    for(study in studyName){
      dir = file.path(dataPath, study)
      sampleName = list.files(dir)
      for(sample in sampleName){
        path = file.path(dir, sample)
        data[[sample]] = list(study=study, sampleName=sample, path=path)
      }
    }
  }else if(mode == "h5"){
    for(study in studyName){
      dir = file.path(dataPath, study)
      sampleName = basename(list.dirs(dir, recursive=FALSE))
      for(sample in sampleName){
        path = file.path(dir, sample, "outs", "filtered_feature_bc_matrix")
        data[[sample]] = list(study=study, sampleName=sample, path=path)
      }
    }
    
  }

  return(data)
}


# 二级函数
loadFile = function(data){
  # input:
  #   data, list
  # change:
  #   读取单细胞数据
  # output:
  #   list

  progress = 0
  progressLen = length(data)
  # 读取单细胞数据
  for(sample in names(data)){
    progress = progress+1
    print(paste0(progress, '/', progressLen))
    files = list.files(data[[sample]][["path"]])
    files = file.path(data[[sample]][["path"]], files)
    barcodesFile = files[grepl(".*barcodes.tsv.*", files)]
    featuresFile = files[grepl(".*features.tsv.*", files)]
    matrixFile = files[grepl(".*matrix.mtx.*", files)]
    dataTemp = tempRead10X(data[[sample]][["path"]],
                           barcodesFile=barcodesFile,
                           featuresFile=featuresFile,
                           matrixFile=matrixFile)
    data[[sample]][["data"]] = CreateSeuratObject(raw.data=dataTemp,
                                                  project=data[[sample]][["sampleName"]])
  }
  
  return(data)
}

In [15]:
# main
if(TRUE){
  data = list()
  # 准备3文件单细胞数据
  data = makeData(dataPath=scRNA_path,
                  studyName=studyNameFile3,
                  mode="3",
                  data=data)
  data = makeData(dataPath=share_path,
                  studyName=studyNameH5,
                  mode="h5",
                  data=data)
  
  # 读取3文件单细胞数据
  data = loadFile(data=data)
  
  # 处理特殊格式的数据
  temp = load("/data1/weiyihu/endometrium/data/raw_data/GSE130560/GSE130560_matrix.RData")
  temp = CreateSeuratObject(raw.data=matrix,
                            project="GSE130560")
  data[["GSE130560"]] = list(study="GSE130560", sampleName="GSE130560", data=temp)
}


[1] "1/13"


ERROR: Error in readMM(file = matrix.loc): could not find function "readMM"


In [11]:
Read10X

function (data.dir, gene.column = 2, cell.column = 1, unique.features = TRUE, 
    strip.suffix = FALSE) 
{
    full.data <- list()
    has_dt <- requireNamespace("data.table", quietly = TRUE) && 
        requireNamespace("R.utils", quietly = TRUE)
    for (i in seq_along(along.with = data.dir)) {
        run <- data.dir[i]
        if (!dir.exists(paths = run)) {
            stop("Directory provided does not exist")
        }
        barcode.loc <- file.path(run, "barcodes.tsv")
        gene.loc <- file.path(run, "genes.tsv")
        features.loc <- file.path(run, "features.tsv.gz")
        matrix.loc <- file.path(run, "matrix.mtx")
        pre_ver_3 <- file.exists(gene.loc)
        if (!pre_ver_3) {
            addgz <- function(s) {
                return(paste0(s, ".gz"))
            }
            barcode.loc <- addgz(s = barcode.loc)
            matrix.loc <- addgz(s = matrix.loc)
        }
        if (!file.exists(barcode.loc)) {
            stop("Barcode file missing. Expecting ", basename(path = barcode.loc))
        }
        if (!pre_ver_3 && !file.exists(features.loc)) {
            stop("Gene name or features file missing. Expecting ", 
                basename(path = features.loc))
        }
        if (!file.exists(matrix.loc)) {
            stop("Expression matrix file missing. Expecting ", 
                basename(path = matrix.loc))
        }
        data <- readMM(file = matrix.loc)
        if (has_dt) {
            cell.barcodes <- as.data.frame(data.table::fread(barcode.loc, 
                header = FALSE))
        }
        else {
            cell.barcodes <- read.table(file = barcode.loc, header = FALSE, 
                sep = "\t", row.names = NULL)
        }
        if (ncol(x = cell.barcodes) > 1) {
            cell.names <- cell.barcodes[, cell.column]
        }
        else {
            cell.names <- readLines(con = barcode.loc)
        }
        if (all(grepl(pattern = "\\-1$", x = cell.names)) & strip.suffix) {
            cell.names <- as.vector(x = as.character(x = sapply(X = cell.names, 
                FUN = ExtractField, field = 1, delim = "-")))
        }
        if (is.null(x = names(x = data.dir))) {
            if (length(x = data.dir) < 2) {
                colnames(x = data) <- cell.names
            }
            else {
                colnames(x = data) <- paste0(i, "_", cell.names)
            }
        }
        else {
            colnames(x = data) <- paste0(names(x = data.dir)[i], 
                "_", cell.names)
        }
        if (has_dt) {
            feature.names <- as.data.frame(data.table::fread(ifelse(test = pre_ver_3, 
                yes = gene.loc, no = features.loc), header = FALSE))
        }
        else {
            feature.names <- read.delim(file = ifelse(test = pre_ver_3, 
                yes = gene.loc, no = features.loc), header = FALSE, 
                stringsAsFactors = FALSE)
        }
        if (any(is.na(x = feature.names[, gene.column]))) {
            warning("Some features names are NA. Replacing NA names with ID from the opposite column requested", 
                call. = FALSE, immediate. = TRUE)
            na.features <- which(x = is.na(x = feature.names[, 
                gene.column]))
            replacement.column <- ifelse(test = gene.column == 
                2, yes = 1, no = 2)
            feature.names[na.features, gene.column] <- feature.names[na.features, 
                replacement.column]
        }
        if (unique.features) {
            fcols = ncol(x = feature.names)
            if (fcols < gene.column) {
                stop(paste0("gene.column was set to ", gene.column, 
                  " but feature.tsv.gz (or genes.tsv) only has ", 
                  fcols, " columns.", " Try setting the gene.column argument to a value <= to ", 
                  fcols, "."))
            }
            rownames(x = data) <- make.unique(names = feature.names[, 
                gene.column])
        }
        if (ncol(x = feature.names)

In [ ]:
# debug
# normalize and identify variable features for each dataset independently
for(sample in names(data)){
  data[[sample]][["data"]] = NormalizeData(data[[sample]][["data"]])
  data[[sample]][["data"]] = FindVariableFeatures(NormalizeData(data[[sample]][["data"]]),
                                                  selection.method="vst",
                                                  nfeatures=2000)
}